In [ ]:
import numpy as np
import pandas as pd

# =====================================
# 0. CONFIG
# =====================================
INPUT_PATH  = r"C:/Users/HP/Downloads/nasdaq100_metrics_ratios (1).csv"
OUTPUT_PATH = r"C:/Users/HP/Downloads/nasdaq100_preprocessed_clean.csv"

Z_THRESH = 1.81
WINSOR_LO = 0.01
WINSOR_HI = 0.99

# =====================================
# 1. LOAD & BASIC CLEAN
# =====================================
df = pd.read_csv(INPUT_PATH)

df.columns = [c.strip() for c in df.columns]
df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")

# =====================================
# 2. DEDUPLICATE (symbol, year)
# Rule: keep FIRST occurrence (file order = snapshot time)
# =====================================
before = len(df)
df = df.sort_index().drop_duplicates(["symbol", "year"], keep="first")
print(f"[Dedup] Dropped {before - len(df)} rows")

# =====================================
# 3. DROP LEAKAGE COLUMNS
# =====================================
leak_cols = [c for c in df.columns if c.endswith("_latest")]
if leak_cols:
    print("[Drop leakage]", leak_cols)
    df = df.drop(columns=leak_cols)

# =====================================
# 4. CREATE LABEL
# =====================================
df["zscore"] = pd.to_numeric(df["zscore"], errors="coerce")
df = df.dropna(subset=["zscore"]).copy()

df["y_distress"] = (df["zscore"] < Z_THRESH).astype(int)
print("[Label] Distress rate:", df["y_distress"].mean().round(4))

# =====================================
# 5. DROP NON-FEATURE COLUMNS
# =====================================
DROP_ALWAYS = ["company", "zscore"]
DROP_ALWAYS = [c for c in DROP_ALWAYS if c in df.columns]

df = df.drop(columns=DROP_ALWAYS)

# =====================================
# 6. IDENTIFY COLUMN TYPES
# =====================================
ID_COLS  = ["symbol", "year"]
CAT_COLS = [c for c in ["sector", "subsector"] if c in df.columns]

NUM_COLS = [
    c for c in df.columns
    if c not in ID_COLS + CAT_COLS + ["y_distress"]
]

# =====================================
# 7. MISSING VALUE IMPUTATION (3-STAGE)
# =====================================

# Ensure numeric
df[NUM_COLS] = df[NUM_COLS].apply(pd.to_numeric, errors="coerce")

# (a) Time interpolation within symbol
df = df.sort_values(["symbol", "year"])
df[NUM_COLS] = (
    df.groupby("symbol")[NUM_COLS]
      .apply(lambda g: g.interpolate(method="linear", limit_direction="both"))
      .reset_index(level=0, drop=True)
)

# (b) Sector-year median
if "sector" in df.columns:
    for col in NUM_COLS:
        if df[col].isna().any():
            med = df.groupby(["sector", "year"])[col].transform("median")
            df[col] = df[col].fillna(med)

# (c) Global median
for col in NUM_COLS:
    if df[col].isna().any():
        df[col] = df[col].fillna(df[col].median())

# =====================================
# 8. WINSORIZATION (OUTLIER CONTROL)
# =====================================
for col in NUM_COLS:
    lo = df[col].quantile(WINSOR_LO)
    hi = df[col].quantile(WINSOR_HI)
    df[col] = df[col].clip(lo, hi)

# =====================================
# 9. FINAL SORT & SANITY CHECK
# =====================================
df = df.sort_values(["symbol", "year"]).reset_index(drop=True)

assert df[NUM_COLS].isna().sum().sum() == 0, "NaN remains in numeric features"

print("[Final]")
print("Rows:", len(df))
print("Years:", df["year"].min(), "→", df["year"].max())
print("Columns:", len(df.columns))

# =====================================
# 10. SAVE CLEAN DATASET
# =====================================
df.to_csv(OUTPUT_PATH, index=False)
print("Saved:", OUTPUT_PATH)
